<a href="https://colab.research.google.com/github/EureXaAI/EurexaBook/blob/main/playground/EureXa_Book_0331.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title 1.  安装依赖库
# 安装 Mujoco: 物理引擎, 广泛用于 RL 和仿真研究
!pip install mujoco
# 安装 Mujoco Mjx: Mujoco 的 JAX 封装, 可微分, 结合 JAX 加速计算
!pip install mujoco_mjx
# 安装 Brax: Google 开发的独立物理引擎, 原生用 JAX 写的
!pip install brax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.3/709.3 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.7 MB/s eta 0:00:00


In [2]:
#@title 2. 检测安装结果
# 从 Colab 中引入文件上传功能（备用）
from google.colab import files

# 引入用于路径和命令行操作的模块
import distutils.util
import os
import subprocess

# 检查是否能够访问 GPU（通过 nvidia-smi 命令）
# 如果失败，就提示用户需要启用 Colab 的 GPU 运行时
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      '无法与 GPU 通信。请确保你使用的是启用了 GPU 的 Colab 运行时。\n'
      '点击上方菜单「运行时」->「更改运行时类型」，并选择 GPU。')

# 添加一个 ICD 配置，让 glvnd 能找到 Nvidia 的 EGL 驱动
# Colab 虽然有 GPU，但驱动不是通过 APT 安装的，ICD 文件默认缺失
# 这段是手动补上该配置文件（详见 NVIDIA 文档）
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# 设置环境变量，告诉 MuJoCo 使用 GPU EGL (headless) 渲染
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# 尝试导入 MuJoCo 并用空 XML 检查安装是否成功
# 如果失败，抛出错误并提醒检查运行时设置或安装输出信息
try:
  print('检查 MuJoCo 安装是否成功:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'MuJoCo 安装出错。请检查上方 shell 输出信息。\n'
      '若使用 Colab，请确保启用了 GPU。\n'
      '点击「运行时」->「更改运行时类型」，启用 GPU。')

print('MuJoCo 安装成功！')

# 设置 XLA 编译器参数，启用 Triton GEMM 以提升训练速度（对某些 GPU 有 30% 提升）
xla_flags = os.environ.get('XLA_FLAGS', '')
xla_flags += ' --xla_gpu_triton_gemm_any=True'
os.environ['XLA_FLAGS'] = xla_flags

Setting environment variable to use GPU rendering:
env: MUJOCO_GL=egl
检查 MuJoCo 安装是否成功:
MuJoCo 安装成功！


In [3]:
#@title 3. 导入用于绘图和图像生成的必要包
# 导入 json 模块，用于处理 JSON 格式的数据（如保存结果或加载配置）
import json

# 导入 itertools，提供高效的迭代器构造工具（如排列组合）
import itertools

# 导入 time 模块，用于计时或设置延时（sleep 等）
import time

# 从 typing 中导入常用的类型注解工具
# Callable（函数类型），List、NamedTuple（结构体），Optional（可空类型），Union（联合类型）
from typing import Callable, List, NamedTuple, Optional, Union

# 导入 numpy，用于数组、矩阵和数值计算
import numpy as np


# ========== 图形绘制和视频支持 ==========

# 打印提示，表示将要安装 mediapy（用于渲染视频）
print("Installing mediapy:")

# 检查 ffmpeg 是否已安装（ffmpeg 用于编码视频）
# 如果未安装，则使用 apt 自动安装 ffmpeg（适用于 Colab）
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)

# 安装 mediapy（用于播放和保存视频/图像）
!pip install -q mediapy

# 导入 mediapy 库（可用于显示仿真结果视频）
import mediapy as media

# 导入 matplotlib 的绘图库，用于绘图（如 reward 曲线）
import matplotlib.pyplot as plt


# ========== numpy 显示优化 ==========

# 设置 numpy 的打印格式：
# precision=3：小数保留 3 位；
# suppress=True：关闭科学计数法；
# linewidth=100：每行最多显示 100 字符，防止换行太多
np.set_printoptions(precision=3, suppress=True, linewidth=100)

Installing mediapy:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.1 MB/s eta 0:00:00


In [4]:
#@title 4. 导入 Mujoco, Mujoco Mjx, Brax 所需包
# 导入 datetime，用于记录训练/评估过程中的时间戳
from datetime import datetime

# functools 提供函数工具，比如偏函数（partial）、缓存（lru_cache）等
import functools

# OS 模块，用于操作系统级别的操作，如路径管理、环境变量等
import os

# 类型注解工具，用于标注函数参数或变量的类型，提高代码可读性和类型检查
from typing import Any, Dict, Sequence, Tuple, Union


# ========== Brax 核心物理模拟模块 ==========

# 导入 Brax 的基础模块，包含基础数据结构和方法
from brax import base

# 导入强化学习环境集合（如 humanoid, ant 等）
from brax import envs

# 导入 Brax 提供的数学工具（如四元数运算、旋转等）
from brax import math

# 从 brax.base 中分别导入三大基础类：
# Base：物理模型的基础结构（包含物体、关节等）
# Motion：表示速度、加速度等动态信息
# Transform：表示位置与方向（位姿）
from brax.base import Base, Motion, Transform

# PipelineState：用于表示 Pipeline 模拟器的状态结构（位置、速度等）
from brax.base import State as PipelineState

# 导入基础环境接口类：
# Env：通用环境接口；
# PipelineEnv：基于物理模拟管线的环境；
# State：用于环境交互中的状态表示
from brax.envs.base import Env, PipelineEnv, State

# brax.io 模块提供输入输出功能：
# html：将环境状态渲染为 HTML 格式（可视化）；
# mjcf：读取 MuJoCo 格式的模型文件；
# model：解析模型结构
from brax.io import html, mjcf, model

# MjxState：MJX 模拟（JAX 版 MuJoCo）中的状态结构
from brax.mjx.base import State as MjxState


# ========== PPO 和 SAC 算法模块 ==========

# 导入 PPO 策略网络构建模块
from brax.training.agents.ppo import networks as ppo_networks

# 导入 PPO 强化学习训练接口
from brax.training.agents.ppo import train as ppo

# 导入 SAC 策略网络构建模块
from brax.training.agents.sac import networks as sac_networks

# 导入 SAC 强化学习训练接口
from brax.training.agents.sac import train as sac


# ========== 工具类库和可视化相关 ==========

# etils.epath：Google 的路径管理库（功能类似 pathlib）
from etils import epath

# flax 是 JAX 生态中的神经网络工具包，struct 用于结构化状态定义
from flax import struct

# orbax 是模型保存与加载工具，orbax_utils 是其辅助工具集
from flax.training import orbax_utils

# IPython 的 HTML 显示模块（在 notebook 中插入交互式视频等）
from IPython.display import HTML, clear_output

# 导入 JAX 核心模块
import jax

# JAX 中的 numpy 子模块（支持自动微分与加速）
from jax import numpy as jp

# matplotlib 的绘图库（用于 reward 曲线等）
from matplotlib import pyplot as plt

# mediapy 用于展示视频、保存渲染结果等
import mediapy as media

# ml_collections.config_dict：用于构建灵活的配置字典（如超参数）
from ml_collections import config_dict


# ========== MuJoCo 模拟器模块 ==========

# 导入原生 MuJoCo 引擎（C 语言实现的物理引擎）
import mujoco

# 从 MuJoCo 导入 MJX：MuJoCo 的 JAX 可微版本（mujoco_mjx）
from mujoco import mjx


# ========== 数值计算基础模块 ==========

# numpy 是通用数值计算库（与 jax.numpy 类似，但不支持 GPU 加速）
import numpy as np

# 导入 orbax 的 checkpoint 模块，用于保存和恢复模型权重
from orbax import checkpoint as ocp

In [5]:
#@title 5. 安装 MuJoCo Playground
# MuJoCo Playground 是一个基于 MuJoCo MJX 构建的, 支持 GPU 加速的强化学习环境套件
!pip install playground

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 94.7 MB/s eta 0:00:00


In [6]:
#@title 6. 导入 Playground 模块（The Playground）

# 从 mujoco_playground 中导入 wrapper 模块：
# 这个模块通常用于封装环境（env）或模型，使其更易于控制和交互，
# 比如包裹 MuJoCo 模型以简化调用或添加自定义控制接口
from mujoco_playground import wrapper

# 从 mujoco_playground 中导入 registry 模块：
# 用于注册和管理自定义的动作、环境、控制器等，
# 通常可以在其中注册自己的“青蛙跳”等动作动作控制逻辑，方便统一调用
from mujoco_playground import registry

mujoco_menagerie not found. Downloading...


Cloning mujoco_menagerie: ██████████| 100/100 [00:26<00:00]


Checking out commit 14ceccf557cc47240202f2354d684eca58ff8de4
Successfully downloaded mujoco_menagerie


In [7]:
#@title 7. 查看 MuJoCo Playground 包含的四足和双足环境
registry.locomotion.ALL_ENVS

('BarkourJoystick',
 'BerkeleyHumanoidJoystickFlatTerrain',
 'BerkeleyHumanoidJoystickRoughTerrain',
 'G1JoystickFlatTerrain',
 'G1JoystickRoughTerrain',
 'Go1JoystickFlatTerrain',
 'Go1JoystickRoughTerrain',
 'Go1Getup',
 'Go1Handstand',
 'Go1Footstand',
 'H1InplaceGaitTracking',
 'H1JoystickGaitTracking',
 'Op3Joystick',
 'SpotFlatTerrainJoystick',
 'SpotGetup',
 'SpotJoystickGaitTracking',
 'T1JoystickFlatTerrain',
 'T1JoystickRoughTerrain')

In [9]:
#@title 8. 训练 Unitree Go1 操纵杆策略
# 操纵杆策略
env_name = 'Go1JoystickFlatTerrain'
# 注册管理环境
env = registry.load(env_name)
# 环境配置
env_cfg = registry.get_default_config(env_name)
# 查看配置
env_cfg

Kd: 0.5
Kp: 35.0
action_repeat: 1
action_scale: 0.5
command_config:
  a:
  - 1.5
  - 0.8
  - 1.2
  b:
  - 0.9
  - 0.25
  - 0.5
ctrl_dt: 0.02
episode_length: 1000
history_len: 1
noise_config:
  level: 1.0
  scales:
    gravity: 0.05
    gyro: 0.2
    joint_pos: 0.03
    joint_vel: 1.5
    linvel: 0.1
pert_config:
  enable: false
  kick_durations:
  - 0.05
  - 0.2
  kick_wait_times:
  - 1.0
  - 3.0
  velocity_kick:
  - 0.0
  - 3.0
reward_config:
  max_foot_height: 0.1
  scales:
    action_rate: -0.01
    ang_vel_xy: -0.05
    dof_pos_limits: -1.0
    energy: -0.001
    feet_air_time: 0.1
    feet_clearance: -2.0
    feet_height: -0.2
    feet_slip: -0.1
    lin_vel_z: -0.5
    orientation: -5.0
    pose: 0.5
    stand_still: -1.0
    termination: -1.0
    torques: -0.0002
    tracking_ang_vel: 0.5
    tracking_lin_vel: 1.0
  tracking_sigma: 0.25
sim_dt: 0.004
soft_joint_pos_limit_factor: 0.95

In [13]:
#@title 9. 采用 PPO 算法
# 从 mujoco_playground 的 config 模块中导入 locomotion_params
# 这个模块中包含为不同环境（如 humanoid、ant 等）预设的强化学习参数配置
from mujoco_playground.config import locomotion_params

# 调用 locomotion_params 中的 brax_ppo_config 函数，
# 根据当前环境名称（env_name）返回对应的 PPO 训练配置（例如学习率、batch 大小等）
ppo_params = locomotion_params.brax_ppo_config(env_name)

# 显示 ppo_params 的内容（通常是一个 config_dict，包含超参数字典）
ppo_params

action_repeat: 1
batch_size: 256
discounting: 0.97
entropy_cost: 0.01
episode_length: 1000
learning_rate: 0.0003
max_grad_norm: 1.0
network_factory:
  policy_hidden_layer_sizes: &id001 !!python/tuple
  - 512
  - 256
  - 128
  policy_obs_key: state
  value_hidden_layer_sizes: *id001
  value_obs_key: privileged_state
normalize_observations: true
num_envs: 8192
num_evals: 10
num_minibatches: 32
num_resets_per_eval: 1
num_timesteps: 200000000
num_updates_per_batch: 4
reward_scaling: 1.0
unroll_length: 20

In [14]:
#@title 10. 域随机化函数: 对摩擦力, 电机, 躯干质心和部件质量等模拟参数进行随机化
registry.get_domain_randomizer(env_name)

<function mujoco_playground._src.locomotion.go1.randomize.domain_randomize(model: mujoco.mjx._src.types.Model, rng: jax.Array)>